In [23]:
# Step 1: Install necessary libraries
!pip install -q transformers datasets accelerate bitsandbytes torch evaluate rouge_score sentencepiece bert_score

In [24]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
import pandas as pd
import random
import evaluate
import warnings
import time

# Suppress warnings to keep the output clean
warnings.filterwarnings("ignore")
from transformers import logging
logging.set_verbosity_error()

In [25]:
# Step 2: Authenticate with Hugging Face
# This is required to download gated models like Llama 3
try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HUGGING_FACE_TOKEN")
except Exception as e:
    print("Could not retrieve Hugging Face token. Please ensure it is stored as a Kaggle secret named 'HUGGING_FACE_TOKEN'.")
    # You can manually paste your token here for local testing if needed:
    # hf_token = "YOUR_HF_TOKEN"
    hf_token = None

In [26]:
# Step 3: Define Model and Dataset Identifiers
dataset_id = "tmnam20/ViMedAQA"

EVAL_FULL_DATASET = False

seed_num = 6
NUM_SAMPLES_INITIAL = 100

# --- ADDED: Boolean toggle for the second randomization ---
# Set to True to get the final 50 samples.
# Set to False to use the initial 200 samples.
ENABLE_SUBSET_SAMPLING = False
NUM_SAMPLES_FINAL = 50

# Step 4: Load and Prepare the Dataset
try:
    dataset = load_dataset(dataset_id, split="test")
    print(f"Dataset loaded successfully! Total samples: {len(dataset)}")

    if EVAL_FULL_DATASET:
        eval_dataset = dataset
    else:
        # --- First Sampling Step: Always get the initial 200 samples ---
        random.seed(seed_num) # for reproducibility
        initial_random_indices = random.sample(range(len(dataset)), NUM_SAMPLES_INITIAL)
        initial_eval_dataset = dataset.select(initial_random_indices)
    
        print(f"Created an initial random evaluation set with {len(initial_eval_dataset)} samples.")
    
        # --- ADDED: Conditional second randomization ---
        if ENABLE_SUBSET_SAMPLING:
            print("Subset sampling is ENABLED. Performing second randomization...")
            # Re-seed to ensure this step is also reproducible
            random.seed(seed_num)
            final_random_indices = random.sample(range(len(initial_eval_dataset)), NUM_SAMPLES_FINAL)
            # Final dataset is the smaller, 50-sample subset
            eval_dataset = initial_eval_dataset.select(final_random_indices)
            print(f"Further randomized and reduced the set to a final size of {len(eval_dataset)} samples.")
        else:
            print("Subset sampling is DISABLED.")
            # Final dataset is the larger, 200-sample set
            eval_dataset = initial_eval_dataset
            print(f"Using the initial set of {len(eval_dataset)} samples for evaluation.")

except Exception as e:
    print(f"Failed to load the dataset. Error: {e}")
    eval_dataset = None

Dataset loaded successfully! Total samples: 2217
Created an initial random evaluation set with 1 samples.
Subset sampling is DISABLED.
Using the initial set of 1 samples for evaluation.


In [27]:
# USER_PROMPTS = {
#     "Extract_EN": "Based on the following Context, extract the direct answer from the text without any additional explanation.",
# }

# SYSTEM_PROMPTS = [
#     (
#         "Expert_SP_EN",
#         "You are a medical expert AI. Based on your expertise, answer the following Question in Vietnamese, using ONLY the provided Context.",
#         "{Extract_EN}\n\n### Context:\n{context}\n\n### Question:\n{question}",
#     ),
# ]

In [28]:
USER_PROMPTS = {
    # "Extract_VI": "Dựa vào Ngữ cảnh sau, trích xuất câu trả lời trực tiếp từ văn bản, không giải thích gì thêm.",
    # "Concise_VI": (
    #     f"Dựa CHỈ vào văn bản trong phần Ngữ cảnh dưới đây, hãy trả lời cho Câu hỏi. "
    #     f"Câu trả lời của bạn phải ngắn gọn, đi thẳng vào vấn đề và không chứa bất kỳ thông tin nào không có trong văn bản. "
    #     f"Không giải thích thêm."
    # ),
    # "Strict_Rules_VI": (
    #     f"TỪ Ngữ cảnh, TRÍCH XUẤT câu trả lời cho Câu hỏi.\n\n"
    #     f"**QUY TẮC:**\n"
    #     f"1. CHỈ sử dụng thông tin từ Ngữ cảnh.\n"
    #     f"2. KHÔNG giải thích các bước của bạn.\n"
    #     f"3. KHÔNG tự suy luận hoặc thêm bất kỳ thông tin bên ngoài nào.\n"
    #     f"4. Cung cấp câu trả lời được trích xuất trực tiếp.\n\n"
    #     f"Dưới đây là Ngữ cảnh và Câu hỏi, hãy đưa câu trả lời TRÍCH XUẤT:"
    # ),

    # --- ENGLISH PROMPTS ---
    "Extract_EN": "Based on the following Context, extract the direct answer from the text without any additional explanation.",
    # "Concise_EN": (
    #     f"Based ONLY on the text in the Context section below, answer the Question. "
    #     f"Your answer must be concise, to the point, and contain no information not present in the text. "
    #     f"Do not add any explanation."
    # ),
    # "Strict_Rules_EN": (
    #     f"From the Context, EXTRACT the answer to the Question.\n\n"
    #     f"**RULES:**\n"
    #     f"1. ONLY use information from the Context.\n"
    #     f"2. DO NOT explain your steps.\n"
    #     f"3. DO NOT infer or add any external information.\n"
    #     f"4. Provide the directly extracted answer.\n\n"
    #     f"Below is the Context and the Question, provide the EXTRACTED answer:"
    # ),

    # "Direct_VI": "Sử dụng Ngữ cảnh sau để trả lời Câu hỏi.",
    # "Direct_EN": "Use the following Context to answer the Question.",

    # "List_VI": (
    #     f"Từ Ngữ cảnh được cung cấp, hãy **liệt kê tất cả** các thông tin dùng để trả lời cho Câu hỏi."
    #     f"Trình bày câu trả lời một cách ngắn gọn, chỉ bao gồm các điểm được tìm thấy."
    # ),
    # "List_EN": (
    #     f"From the provided Context, **list all** the information that answers the Question. "
    #     f"Present the answer concisely, including only the points found."
    # ),

    "ViMedAQA": "",
}

default_instruction_format_vi = "{base_instruction}\n\n### Ngữ cảnh:\n{context}\n\n### Câu hỏi:\n{question}"
default_instruction_format_en = "{base_instruction}\n\n### Context:\n{context}\n\n### Question:\n{question}"

SYSTEM_PROMPTS = [
    # (
    #     "Expert_SP_VI",
    #     "Bạn là một AI chuyên gia y tế. Dựa trên kiến thức chuyên môn của mình, hãy trả lời Câu hỏi sau CHỈ dựa vào Ngữ cảnh được cung cấp.",
    #     default_instruction_format_vi,
    # ),
    (
        "Expert_SP_EN",
        "You are a medical expert AI. Based on your expertise, answer the following Question in Vietnamese, using ONLY the provided Context.",
        default_instruction_format_en,
    ),

    # (
    #     "Default_SP_VI",
    #     "Bạn là một trợ lý y tế hữu ích. Hãy trả lời Câu hỏi sau CHỈ dựa vào Ngữ cảnh được cung cấp.",
    #     default_instruction_format_vi,
    # ),
    # (
    #     "Default_SP_EN",
    #     "You are a helpful medical assistant. Answer the following Question in Vietnamese, based ONLY on the provided Context.",
    #     default_instruction_format_en,
    # ),

    # (
    #     "Empty_SP_VI",
    #     "",
    #     default_instruction_format_vi,
    # ),
    # (
    #     "Empty_SP_EN",
    #     "",
    #     default_instruction_format_en,
    # ),

    (
        "ViMedAQA_SP_VI",
        "Dựa vào ngữ cảnh sau và kiến thức của bạn, trả lời câu hỏi sau bằng tiếng Việt.",
        default_instruction_format_vi,
    ),
    (
        "ViMedAQA_SP_EN",
        "Based on the following context and your knowledge, answer the following question in Vietnamese.",
        default_instruction_format_en,
    ),
]

FIXED_EXPERIMENT_PAIRINGS = [
    ("Extract_EN", "Expert_SP_EN"),
    ("ViMedAQA", "ViMedAQA_SP_VI"),
    ("ViMedAQA", "ViMedAQA_SP_EN"),
]

# Mode 1: Run all strategies defined in USER_PROMPTS (False)
# Mode 2: Run only the single, specified strategy for a targeted comparison (True)
USE_BEST_PROMPT_ONLY = False
BEST_STRATEGY_NAME = "Extract_VI" # Specify the prompt to use in Mode 2
PRINT_PROMPT = False

model_ids = [
    "arcee-ai/Arcee-VyLinh",
    # "vilm/vietcuna-3b-v2",
    # "alpha-ai/LLAMA3-3B-Medical-COT",
    
    # "vilm/vinallama-2.7b-chat",
    # "sail/Sailor-4B",
]

In [29]:
# --- UPDATED: Step 1 - Define Bilingual Prompt Engineering Strategies ---
generation_times = {}

def create_prompt_and_get_config(model_id, tokenizer, system_prompt, full_instruction_text):
    """
    A single, unified function to create a model-specific prompt and return
    the associated answer start tag for parsing.

    Returns:
        tuple: (formatted_prompt_string, answer_start_tag_string)
    """  
    # Models WITHOUT a dedicated System Prompt
    # For these, we combine the system and user prompts into a single instruction.
    if model_id in ["vilm/vietcuna-3b-v2", "sail/Sailor-4B"]:
        # If a system prompt is provided, prepend it.
        if system_prompt:
            combined_instruction = f"{system_prompt}\n\n{full_instruction_text}"
        # Otherwise, just use the user instruction directly.
        else:
            combined_instruction = full_instruction_text

        if model_id == "vilm/vietcuna-3b-v2":
            prompt = f"A chat between a curious user and an artificial intelligence assistant.\nUSER: {combined_instruction}\nASSISTANT:"
            answer_start_tag = "ASSISTANT:"
            return prompt, answer_start_tag

        # https://huggingface.co/sail/Sailor-4B
        if model_id == "sail/Sailor-4B":
            prompt = f"{combined_instruction}\n\nCâu trả lời:"
            answer_start_tag = ""
            return prompt, answer_start_tag

    # For chat models, we build the message list programmatically.
    messages = []
    # Only add the system role if the system_prompt is not empty.
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    
    # Add the user/question role
    if model_id == "sail/Sailor-4B-Chat":
        messages.append({"role": "question", "content": full_instruction_text})
    else:
        # Default to "user" role for all other chat models
        messages.append({"role": "user", "content": full_instruction_text})
    
    if model_id == "arcee-ai/Arcee-VyLinh":
        prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        answer_start_tag = "<|im_start|>assistant"
        return prompt, answer_start_tag

    if model_id == "alpha-ai/LLAMA3-3B-Medical-COT":
        prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        answer_start_tag = "<|start_header_id|>assistant<|end_header_id|>"
        return prompt, answer_start_tag
    
    # https://huggingface.co/sail/Sailor-4B-Chat
    if model_id == "sail/Sailor-4B-Chat":
        prompt = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        answer_start_tag = "answer:"
        return prompt, answer_start_tag

    if "vilm/vinallama-2.7b" in model_id:
        # This model's template is custom and doesn't use apply_chat_template,
        # so we handle it separately.
        if system_prompt:
            prompt = (
                f"<|im_start|>system\n{system_prompt}<|im_end|>\n"
                f"<|im_start|>user\n{full_instruction_text}<|im_end|>\n"
                f"<|im_start|>assistant"
            )
        else:
            # Version without a system prompt
            prompt = (
                f"<|im_start|>user\n{full_instruction_text}<|im_end|>\n"
                f"<|im_start|>assistant"
            )
        answer_start_tag = "<|im_start|>assistant"
        return prompt, answer_start_tag

    # Nothing matches
    return "", ""

In [30]:
# Step 5: Generate Answers from Each Model
all_generated_answers = {}
intermediate_results = {} 

if eval_dataset and hf_token:
    # Wrap each answer in a list to create the required List[List[str]] structure
    ground_truth_answers = [[sample['answer']] for sample in eval_dataset] 
    questions = [sample['question'] for sample in eval_dataset]

    wide_results = []
    for i, sample in enumerate(eval_dataset):
        wide_results.append({
            "Sample_ID": i,
            "Question": sample['question'],
            "Context": sample['context'],
            "Ground_Truth_Answer": ground_truth_answers[i][0]
        })
    
    # Loop through each model to generate answers
    for model_id in model_ids:
        print("\n" + "="*50)
        print(f"Loading model: {model_id}")
        print("="*50)

        model, tokenizer, text_generator = None, None, None

        try:
            # Load the tokenizer and model with 4-bit quantization to save memory
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
                bnb_4bit_use_double_quant=False,
            )
            tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                token=hf_token,
                quantization_config=bnb_config, # <-- PASS THE CONFIG OBJECT HERE
                device_map="auto",
                trust_remote_code=True
            )

            # Set up the text generation pipeline
            text_generator = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
            )

            experiments_to_run = []
            used_user_prompts = set()
            used_system_prompts = set()
            # Create a dictionary for quick lookups of system prompts
            system_prompts_map = {sp[0]: sp for sp in SYSTEM_PROMPTS}
            
            user_prompts_to_run = list(USER_PROMPTS.keys()) if not USE_BEST_PROMPT_ONLY else [BEST_STRATEGY_NAME]

            # 2. Process your fixed pairings first.
            print("--- Processing Fixed Experiment Pairings ---")
            for prompt_name, sp_name in FIXED_EXPERIMENT_PAIRINGS:
                # Ensure the prompts exist before pairing
                if prompt_name not in USER_PROMPTS:
                    print(f"Warning: Fixed user prompt '{prompt_name}' not found. Skipping.")
                    continue
                if sp_name not in system_prompts_map:
                    print(f"Warning: Fixed system prompt '{sp_name}' not found. Skipping.")
                    continue
                
                # Retrieve details for the system prompt
                _, system_prompt_content, instruction_format = system_prompts_map[sp_name]
                
                # Add the specific pair to the list of experiments
                experiments_to_run.append((prompt_name, sp_name, system_prompt_content, instruction_format))
                
                # Mark these prompts as "used" so they aren't paired again
                used_user_prompts.add(prompt_name)
                used_system_prompts.add(sp_name)
                print(f"  + Added fixed pair: ('{prompt_name}', '{sp_name}')")

            # 3. Automatically pair the remaining (unused) prompts.
            print("\\n--- Processing Automatic Remaining Pairings ---")
            for sp_name, system_prompt, instruction_format in SYSTEM_PROMPTS:
                # If this system prompt was already in a fixed pair, skip it
                if sp_name in used_system_prompts:
                    continue
                
                # If the format is a template, create a combination for each task instruction
                if "{base_instruction}" in instruction_format:
                    for prompt_name in user_prompts_to_run:
                        # Only pair English prompts with English strategies, and VI with VI
                        if (sp_name.endswith("_EN") and prompt_name.endswith("_EN")) or \
                           (sp_name.endswith("_VI") and prompt_name.endswith("_VI")):
                            experiments_to_run.append((prompt_name, sp_name, system_prompt, instruction_format))
                # If it's self-contained, add it just once
                else:
                    experiments_to_run.append(("Self-Contained", sp_name, system_prompt, instruction_format))

            for prompt_name, sp_name, system_prompt, instruction_format in experiments_to_run:
                print(f"\n===== Running Experiment: [Task: {prompt_name}] | [System Prompt: {sp_name}] =====")
                # Fetch the base instruction, providing an empty string for the "Self-Contained" case
                base_instruction = USER_PROMPTS.get(prompt_name, "")
                result_key_tuple = (model_id, prompt_name, sp_name)
                prompts = []
                answer_start_tag = ""
            
                for sample in eval_dataset:
                    # If the base instruction is empty (our special case), use a cleaner format.
                    if not base_instruction:
                        if sp_name.endswith("_EN"):
                            full_instruction_text = (
                                f"### Context:\n{sample['context']}\n\n"
                                f"### Question:\n{sample['question']}"
                            )
                        else: # Default to Vietnamese
                            full_instruction_text = (
                                f"### Ngữ cảnh:\n{sample['context']}\n\n"
                                f"### Câu hỏi:\n{sample['question']}"
                            )
                    # Otherwise, use the standard template format.
                    else:
                        full_instruction_text = instruction_format.format(
                            base_instruction=base_instruction, 
                            context=sample['context'], 
                            question=sample['question']
                        )
                    
                    prompt, tag = create_prompt_and_get_config(model_id, tokenizer, system_prompt, full_instruction_text)
                    if PRINT_PROMPT:
                        print(prompt)
                    
                    prompts.append(prompt)
                    if not answer_start_tag: answer_start_tag = tag
            
                if prompts == "" and answer_start_tag == "":
                    print("No model matches")
                    break

                start_time = time.time()

                # print(f"Generating answers for {len(prompts)} prompts using {model_id} with '{prompt_name}' strategy...")
                # Generate answers for the entire batch
                generated_outputs_batch = text_generator(
                    prompts,
                    max_new_tokens=256,
                    do_sample=False,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.eos_token_id,
                )

                end_time = time.time()
                generation_time = end_time - start_time
                generation_times[result_key_tuple] = generation_time # Lưu thời gian đã đo
    
                # Extract the clean answers
                model_answers = []
                
                for i, output in enumerate(generated_outputs_batch):
                    generated_text = output[0]['generated_text']
                    # Check if the tag is not empty AND exists in the text before splitting
                    if answer_start_tag and answer_start_tag in generated_text:
                        clean_answer = generated_text.split(answer_start_tag)[-1].strip()
                    else:
                        # Fallback for base models or if the tag isn't found
                        clean_answer = generated_text.replace(prompts[i], "").strip()

                    if prompt_name == "Extract_EN":
                        redundant_prefix = "Direct answer:\n"
                        if clean_answer.startswith(redundant_prefix):
                            clean_answer = clean_answer[len(redundant_prefix):].strip()
                    
                    model_answers.append(clean_answer)
    
                all_generated_answers[result_key_tuple] = model_answers
                
                answer_column_name = f"{prompt_name}"
                for i, answer in enumerate(model_answers):
                    # The key identifies a unique row in the final table
                    row_key = (i, model_id, sp_name)
                    
                    # If this is the first time we see this row, initialize it with static info
                    if row_key not in intermediate_results:
                        intermediate_results[row_key] = {
                            "Sample_ID": i,
                            "Question": eval_dataset[i]['question'],
                            "Context": eval_dataset[i]['context'],
                            "Ground_Truth_Answer": ground_truth_answers[i][0],
                            "Model": model_id,
                            "System_Prompt": sp_name
                        }
                    
                    # Add the generated answer to the correct column for this row
                    intermediate_results[row_key][answer_column_name] = answer

                print(f"Time for generating answer: {generation_time:.2f} seconds.")

        except Exception as e:
            print(f"An error occurred while processing {model_id}: {e}")
        finally:
            # Check if variables were successfully created before deleting
            if model is not None: del model
            if tokenizer is not None: del tokenizer
            if text_generator is not None: del text_generator
            torch.cuda.empty_cache()

            import gc
            gc.collect()

else:
    print("Skipping generation due to issues with the dataset or Hugging Face token.")


Loading model: arcee-ai/Arcee-VyLinh


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

--- Processing Fixed Experiment Pairings ---
  + Added fixed pair: ('Extract_EN', 'Expert_SP_EN')
  + Added fixed pair: ('ViMedAQA', 'ViMedAQA_SP_VI')
  + Added fixed pair: ('ViMedAQA', 'ViMedAQA_SP_EN')
\n--- Processing Automatic Remaining Pairings ---

===== Running Experiment: [Task: Extract_EN] | [System Prompt: Expert_SP_EN] =====
<|im_start|>system
You are a medical expert AI. Based on your expertise, answer the following Question in Vietnamese, using ONLY the provided Context.<|im_end|>
<|im_start|>user
Based on the following Context, extract the direct answer from the text without any additional explanation.

### Context:
Khi quá liều Alzental Shinpoong, bệnh nhân sẽ có triệu chứng như phản ứng sốc phản vệ, phát ban, rụng tóc nhiều. Nhiều biểu hiện nghiêm trọng hơn nhiều như suy giảm bạch cầu, suy giảm chức năng gan.4 Do đó, hãy đến bệnh viện ngay lập tức và mang theo thuốc để bác sĩ có thể tìm lí do dễ dàng và xử trí kịp thời.

### Question:
Tại sao cần đưa bệnh nhân đến bệnh 

In [31]:
# --- Step 5.5 - Assemble and Save Results in Hybrid-Format CSV File ---
if intermediate_results:
    print("\n" + "="*50)
    print("Assembling results into the desired hybrid format...")
    print("="*50)
    
    final_results_list = list(intermediate_results.values())
    results_df_hybrid = pd.DataFrame(final_results_list)

    # --- Define a clear column order for the final CSV ---
    static_columns = [
        "Sample_ID", "Model", "System_Prompt", 
        "Question", "Context", "Ground_Truth_Answer"
    ]
    
    # Dynamically create the answer column names based on the strategies that were run
    user_prompts_to_run = list(USER_PROMPTS.keys()) if not USE_BEST_PROMPT_ONLY else [BEST_STRATEGY_NAME]
    answer_columns = sorted([f"{p_name}" for p_name in user_prompts_to_run])
    
    # Add the special "Self-Contained" column if it exists in the dataframe
    if "Self-Contained" in results_df_hybrid.columns:
        answer_columns.append("Self-Contained")
    
    # Combine and reorder the DataFrame, handling missing columns gracefully
    final_column_order = static_columns + answer_columns
    # Ensure we only try to order by columns that actually exist in the dataframe
    existing_columns_in_order = [col for col in final_column_order if col in results_df_hybrid.columns]
    results_df_hybrid = results_df_hybrid[existing_columns_in_order]

    results_df_hybrid = results_df_hybrid.sort_values(by=["Sample_ID", "Model", "System_Prompt"]).reset_index(drop=True)

    output_file_path = "/kaggle/working/results.csv"
    results_df_hybrid.to_csv(output_file_path, index=False, encoding='utf-8-sig')
    
    print(f"Complete! Saved {len(results_df_hybrid)} rows to the file:")
    print(output_file_path)
    
    display(results_df_hybrid.head())
    
else:
    print("\nNo results were generated to save.")


Assembling results into the desired hybrid format...
Complete! Saved 3 rows to the file:
/kaggle/working/results.csv


,Sample_ID,Model,System_Prompt,Question,Context,Ground_Truth_Answer,Extract_EN,ViMedAQA
0,0,arcee-ai/Arcee-VyLinh,Expert_SP_EN,Tại sao cần đưa bệnh nhân đến bệnh viện ngay k...,"Khi quá liều Alzental Shinpoong, bệnh nhân sẽ ...",Việc đưa bệnh nhân đến bệnh viện ngay khi dùng...,Để bác sĩ có thể tìm lí do dễ dàng và xử trí k...,NaN
1,0,arcee-ai/Arcee-VyLinh,ViMedAQA_SP_EN,Tại sao cần đưa bệnh nhân đến bệnh viện ngay k...,"Khi quá liều Alzental Shinpoong, bệnh nhân sẽ ...",Việc đưa bệnh nhân đến bệnh viện ngay khi dùng...,NaN,Khi bệnh nhân dùng quá liều Alzental Shinpoong...
2,0,arcee-ai/Arcee-VyLinh,ViMedAQA_SP_VI,Tại sao cần đưa bệnh nhân đến bệnh viện ngay k...,"Khi quá liều Alzental Shinpoong, bệnh nhân sẽ ...",Việc đưa bệnh nhân đến bệnh viện ngay khi dùng...,NaN,Khi bệnh nhân dùng quá liều Alzental Shinpoong...


In [32]:
pd.set_option('display.max_rows', None)

# Step 6: Evaluate the Generated Answers
if all_generated_answers:
    # Load all the metrics we need
    rouge_metric = evaluate.load('rouge')
    bleu_metric = evaluate.load('bleu')
    meteor_metric = evaluate.load('meteor')
    bertscore_metric = evaluate.load('bertscore')

    evaluation_results = []

    print("\n" + "="*50)
    print("Calculating Evaluation Metrics")
    print("="*50)

    for result_key, predictions in all_generated_answers.items():
        model_name, prompt_name, sp_name = result_key
        # Check for empty predictions to prevent ZeroDivisionError in BLEU ---
        # The `any()` function returns False if all strings in the list are empty.
        if not any(predictions):
            print(f"  WARNING: Model & User Prompt '{result_key}' produced empty answers for all samples. Assigning all metric scores to 0.")
            result_row = {
                "Model & User Prompt": result_key,
                "ROUGE-L": 0.0,
                "BLEU": 0.0,
                "METEOR": 0.0,
                "BERTScore-F1": 0.0,
                "Avg-Score": 0.0
            }
            evaluation_results.append(result_row)
            # Use `continue` to skip the rest of the loop and move to the next model
            continue
    
        # If predictions are valid, compute metrics as normal
        rouge_scores = rouge_metric.compute(predictions=predictions, references=ground_truth_answers)
        bleu_scores = bleu_metric.compute(predictions=predictions, references=ground_truth_answers)
        meteor_scores = meteor_metric.compute(predictions=predictions, references=ground_truth_answers)
        bertscore_scores = bertscore_metric.compute(predictions=predictions, references=ground_truth_answers, lang="vi")
        
        # Calculate individual scores for the current result_key
        rouge_l = round(rouge_scores['rougeL'], 4)
        bleu = round(bleu_scores['bleu'], 4)
        meteor = round(meteor_scores['meteor'], 4)
        bertscore_f1 = round(sum(bertscore_scores['f1']) / len(bertscore_scores['f1']), 4)

        # Calculate the average score
        avg_score = round((rouge_l + bleu + meteor + bertscore_f1) / 4, 4)
    
        # Store results (this part is the same as before)
        result_row = {
            "Model": model_name,
            "User_Prompt": prompt_name,
            "System_Prompt": sp_name,
            "ROUGE-L": rouge_l,
            "BLEU": bleu,
            "METEOR": meteor,
            "BERTScore-F1": bertscore_f1,
            "Avg-Score": avg_score,
            "Generation Time (s)": round(generation_times.get(result_key, 0), 2),
        }
        evaluation_results.append(result_row)

    # Step 7: Display Results
    results_df = pd.DataFrame(evaluation_results)
    # Sort for better comparison
    results_df = results_df.sort_values(by="Avg-Score", ascending=False).reset_index(drop=True)
    print("\n--- Comparative Evaluation Results ---")
    display(results_df)

    # Save the evaluation results DataFrame to a separate CSV file
    evaluation_output_path = "/kaggle/working/eval_results.csv"
    results_df.to_csv(evaluation_output_path, index=False, encoding='utf-8-sig')
    print(f"\nEvaluation results successfully saved to: {evaluation_output_path}")

else:
    print("\nNo answers were generated. Skipping evaluation.")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!



Calculating Evaluation Metrics

--- Comparative Evaluation Results ---


,Model,User_Prompt,System_Prompt,ROUGE-L,BLEU,METEOR,BERTScore-F1,Avg-Score,Generation Time (s)
0,arcee-ai/Arcee-VyLinh,ViMedAQA,ViMedAQA_SP_EN,0.4072,0.1475,0.7121,0.8029,0.5174,8.28
1,arcee-ai/Arcee-VyLinh,ViMedAQA,ViMedAQA_SP_VI,0.3235,0.1422,0.6396,0.7888,0.4735,13.87
2,arcee-ai/Arcee-VyLinh,Extract_EN,Expert_SP_EN,0.1519,0.0000,0.0680,0.6877,0.2269,2.04



Evaluation results successfully saved to: /kaggle/working/eval_results.csv


In [33]:
    # "Few_Shot_VI": (
    #     f"Dựa vào các Ví dụ sau đây, hãy trả lời Câu hỏi cuối cùng bằng cách trích xuất thông tin từ Ngữ cảnh được cung cấp.\n\n"
    #     f"--- Ví dụ 1 ---\n"
    #     f"Ngữ cảnh: Thuốc Biviantac được chỉ định để điều trị các trường hợp do tăng tiết acid quá mức như: - Khó tiêu, nóng rát hay đau vùng thượng vị. - Trướng bụng, đầy hơi, ợ nóng, ợ hơi hay ợ chua. - Tăng độ acid, đau rát dạ dày. - Các rối loạn thường gặp trong những bệnh lý loét dạ dày tá tràng, thực quản.\n"
    #     f"Câu hỏi: Biviantac có thể điều trị trướng bụng, đầy hơi không?\n"
    #     f"Câu trả lời: Có, Biviantac có thể điều trị các tình trạng như trướng bụng, đầy hơi, ợ nóng, ợ hơi hay ợ chua.\n\n"
    #     f"--- Ví dụ 2 ---\n"
    #     f"Ngữ cảnh: Thuốc Atorvastatin T.V Pharm được dùng đường uống.\n"
    #     f"Câu hỏi: Tổng hợp các cách dùng hiệu quả để quản lý Atorvastatin T.V Pharm?\n"
    #     f"Câu trả lời: Các cách thức dùng thuốc Atorvastatin T.V Pharm hiệu quả là sử dụng đường uống.\n\n"
    #     f"--- Ví dụ 3 ---\n"
    #     f"Ngữ cảnh: - Buồn nôn, nôn, khó tiêu, khó chịu ở thượng vị, ợ nóng, đau dạ dày, loét dạ dày – ruột. - Mệt mỏi. - Ban, mày đay. - Thiếu máu tan huyết. - Yếu cơ. - Khó thở, sốc phản vệ.\n"
    #     f"Câu hỏi: Các tác dụng phụ thường gặp của thuốc Aspirin 81 là gì?\n"
    #     f"Câu trả lời: Các tác dụng phụ thường gặp của thuốc Aspirin 81 bao gồm buồn nôn, nôn, khó tiêu, khó chịu ở thượng vị, ợ nóng, đau dạ dày, loét dạ dày – ruột.\n\n"
    #     f"--- Ví dụ 4 ---\n"
    #     f"Ngữ cảnh: Các chị em có thể thỉnh thoảng thấy kinh nguyệt ra nhiều hoặc ra máu giữa các kỳ kinh (chảy máu giữa kỳ kinh nguyệt).\n"
    #     f"Câu hỏi: Các chị em có thể gặp tình trạng rong kinh không?\n"
    #     f"Câu trả lời: Có.\n\n"
    #     f"--- Bây giờ, hãy trả lời Câu hỏi sau dựa trên Ngữ cảnh của nó---"
    # ),
    # "Full_VI": (
    #     f"Dựa vào Ngữ cảnh sau, hãy trích xuất câu trả lời **đầy đủ và toàn diện nhất** có thể từ văn bản."
    #     f"Đảm bảo rằng bạn đã bao gồm **tất cả** các điểm có liên quan để trả lời cho câu hỏi."
    # ),

    # "Full_EN": (
    #     f"Based on the following Context, extract the most **complete and comprehensive** answer possible from the text. "
    #     f"Ensure you have included **all** relevant points to answer the question."
    # ),

    # "ViMedAQA": "",

    # (
    #     "ViMedAQA_SP_VI",
    #     "Dựa vào ngữ cảnh sau và kiến thức của bạn, trả lời câu hỏi sau bằng tiếng Việt",
    #     "### Ngữ cảnh:\n{context}\n\n### Câu hỏi:\n{question}",
    # ),
    # (
    #     "ViMedAQA_SP_EN",
    #     "Based on the following context and your knowledge, answer the following question in Vietnamese.",
    #     "### Context:\n{context}\n\n### Question:\n{question}",
    # ),